In [2]:
from aeon.classification.distance_based import KNeighborsTimeSeriesClassifier
from aeon.datasets import load_classification
from aeon.datasets import load_unit_test
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

In [3]:
X_train_f, y_train_f = load_classification(name="KeplerLightCurves", split="train", return_metadata=False)
X_test_f, y_test_f = load_classification(name="KeplerLightCurves", split="test", return_metadata=False)

In [193]:
X_train = X_train_f.squeeze(axis=1)
y_train = y_train_f.astype(int)
X_test = X_test_f.squeeze(axis=1)
y_test = y_test_f.astype(int)

X_train_window = X_train[:, :500]   
X_test_window = X_test[:, :500]

In [194]:
class_a_tr_idxs = np.argwhere(y_train == 3).flatten()
x_tr_class_a = X_train_window[class_a_tr_idxs, :]
class_b_tr_idxs = np.argwhere(y_train == 5).flatten()
x_tr_class_b = X_train_window[class_b_tr_idxs, :]
X_train_sub = np.vstack([x_tr_class_a, x_tr_class_b])
y_train_sub = np.concatenate([np.ones(len(class_a_tr_idxs), dtype=np.int8), 2*np.ones(len(class_b_tr_idxs), dtype=np.int8)])

In [195]:
class_a_te_idxs = np.argwhere(y_test == 3).flatten()
x_te_class_a = X_test_window[class_a_te_idxs, :]
class_b_te_idxs = np.argwhere(y_test == 5).flatten()
x_te_class_b = X_test_window[class_b_te_idxs, :]
X_test_sub = np.vstack([x_te_class_a, x_te_class_b])
y_test_sub = np.concatenate([np.ones(len(class_a_te_idxs), dtype=np.int8), 2*np.ones(len(class_b_te_idxs), dtype=np.int8)])

In [196]:
print(X_train_sub.shape)
print(y_train_sub.shape)
print(X_test_sub.shape)
print(y_test_sub.shape)

(427, 500)
(427,)
(185, 500)
(185,)


In [197]:
# rescale the data
zs = StandardScaler().fit(X_train_sub)
X_train_sub_zs = zs.transform(X_train_sub)  
X_test_sub_zs = zs.transform(X_test_sub)

In [201]:
classifier = KNeighborsTimeSeriesClassifier(distance="dtw")
classifier.fit(X_train_sub_zs, y_train_sub)
sc = classifier.score(X_test_sub_zs, y_test_sub)

In [202]:
sc

0.918918918918919